# Реализация случайного леса

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

np.random.seed(42)

N_ESTIMATORS = 100   # Пример гиперпараметра, который вы можете настроить
MAX_DEPTH = 5        # Пример гиперпараметра
SUBSPACE_DIM = 2     # Пример гиперпараметра

class random_forest(object):
    def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state

        np.random.seed(self.random_state)
        self._estimators = []
        self.subspace_idx = []  # Для хранения индексов признаков, выбранных для каждого дерева

    def fit(self, X, y):
        n_samples, n_features = X.shape

        for i in range(self.n_estimators):
            # Выбор подпространства
            features_idx = np.random.choice(n_features, self.subspaces_dim, replace=False)
            self.subspace_idx.append(features_idx)

            # Сэмплирование данных с возвращением
            sample_indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sampled = X[sample_indices][:, features_idx]
            y_sampled = y[sample_indices]

            # Обучение дерева
            tree = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X_sampled, y_sampled)
            self._estimators.append(tree)

    def predict(self, X):
        # Формируем массив для хранения предсказаний
      predictions = np.zeros((X.shape[0], self.n_estimators))

      for i in range(self.n_estimators):
          # Получаем предсказания для текущего дерева и сохраняем их
          features_idx = self.subspace_idx[i]
          X_subspace = X[:, features_idx]
          predictions[:, i] = self._estimators[i].predict(X_subspace)

      # Приводим предсказания к целочисленному типу перед расчетом мажоритарного голосования
      predictions = predictions.astype(int)

      # Возвращаем мажоритарное голосование
      return np.array([np.bincount(predictions[i]).argmax() for i in range(predictions.shape[0])])

# Пример использования:
if __name__ == "__main__":
    # Загружаем датасет Ирисов Фишера
    iris = load_iris()
    X = iris.data
    y = iris.target

    # Разделяем выборку на обучающую и тестовую
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Создаем и обучаем экземпляр random_forest
    forest = random_forest(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, subspaces_dim=SUBSPACE_DIM, random_state=42)
    forest.fit(X_train, y_train)

    # Получаем предсказания и рассчитываем точность
    y_pred = forest.predict(X_test)
    accuracy = np.mean(y_pred == y_test)

    # Выводим точность
    print(accuracy)  # Убедитесь, что вы можете увидеть точность


1.0


## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [ ]:
X

array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])

In [ ]:
X.shape

(5, 7)

In [ ]:
y.shape

(5,)

In [ ]:
forest = random_forest(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, subspaces_dim=SUBSPACE_DIM, random_state=42)
forest.fit(X_train, y_train)

In [ ]:
preds = rf.predict(X)

In [ ]:
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])